<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled164.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===================================================================
# === Step 1: Install Libraries and Import Dependencies
# ===================================================================
print("--> Step 1: Installing necessary libraries...")
# Use -q for a quieter installation
%pip install -q openmm
%pip install -q pdbfixer
%pip install -q mdtraj

# Import the libraries we will use
import openmm as mm
from openmm import app, unit
import mdtraj as md
import matplotlib.pyplot as plt
import numpy as np
from sys import stdout

print("--> Libraries installed successfully.")


# ===================================================================
# === Step 2: Prepare the Molecular System
# ===================================================================
print("\n--> Step 2: Preparing the molecular system...")

# Download a sample RNA hairpin structure (PDB ID: 2KOC) as a stand-in
!wget -q -O rna_hairpin.pdb https://files.rcsb.org/download/2KOC.pdb

# Use PDBFixer to prepare the molecule (e.g., add missing hydrogen atoms)
fixer = app.PDBFixer(filename='rna_hairpin.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True) # Remove non-RNA atoms
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0) # Assume a pH of 7.0

# Define the force field (physics rules) for RNA and water
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# Create the system by adding a solvent (water) box around the RNA
modeller = app.Modeller(fixer.topology, fixer.positions)
modeller.addSolvent(forcefield, model='tip3pfb', padding=1.0*unit.nanometer)

# Create the final system for OpenMM to simulate
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME,
                                 nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)

print("--> RNA structure has been prepared and solvated in a water box.")


# ===================================================================
# === Step 3: Configure and Run the MD Simulation
# ===================================================================
print("\n--> Step 3: Configuring and running the simulation...")

# Configure the simulation settings (integrator, platform)
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
platform = mm.Platform.getPlatformByName('CUDA') # Use the GPU
properties = {'CudaPrecision': 'mixed'} # Use mixed precision for performance
simulation = app.Simulation(modeller.topology, system, integrator, platform, properties)
simulation.context.setPositions(modeller.positions)

# --- 1. Minimize Energy ---
print('Minimizing energy of the system...')
simulation.minimizeEnergy()

# --- 2. Equilibrate the System ---
print('Equilibrating the system at 300K...')
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
simulation.step(5000) # Run equilibration for 10 ps

# --- 3. Run Production Simulation ---
print('Running Production MD...')
# Set up reporters to save the trajectory (the "movie") and log data
simulation.reporters.append(app.DCDReporter('trajectory_undamaged.dcd', 5000))
simulation.reporters.append(app.StateDataReporter(stdout, 5000, step=True,
        potentialEnergy=True, temperature=True, progress=True, remainingTime=True, speed=True, totalSteps=100000))

# Run the production simulation for 200 picoseconds
simulation.step(100000)

print('\n--> Simulation complete!')


# ===================================================================
# === Step 4: Analyze the Results
# ===================================================================
print("\n--> Step 4: Analyzing the simulation trajectory...")

# Load the trajectory file we just created
traj_undamaged = md.load('trajectory_undamaged.dcd', top=app.PDBFile(fixer.topology, fixer.positions).topology)

# Calculate the Root Mean Square Deviation (RMSD) from the starting frame
rmsd_undamaged = md.rmsd(traj_undamaged, traj_undamaged, frame=0)

# Plot the RMSD over time
plt.figure(figsize=(12, 7))
plt.plot(traj_undamaged.time, rmsd_undamaged, label='Undamaged RNA (Cautious Strategy)')
plt.xlabel('Time (ps)', fontsize=12)
plt.ylabel('RMSD (nm)', fontsize=12)
plt.title('Structural Stability of the RNA Molecule', fontsize=14)
plt.legend()
plt.grid(True)
plt.show()

print("\n--> Analysis complete. A stable RMSD indicates the structure is not unfolding.")


--> Step 1: Installing necessary libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 122.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement pdbfixer (from versions: none)
ERROR: No matching distribution found for pdbfixer
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 7.6 MB/s eta 0:00:00
--> Libraries installed successfully.

--> Step 2: Preparing the molecular system...


AttributeError: module 'openmm.app' has no attribute 'PDBFixer'

In [ ]:
# Install PDBFixer separately
%pip install -q pdbfixer

ERROR: Could not find a version that satisfies the requirement pdbfixer (from versions: none)
ERROR: No matching distribution found for pdbfixer


In [ ]:
# ===================================================================
# === Step 1: Install Libraries and Import Dependencies
# ===================================================================
print("--> Step 1: Installing necessary libraries...")
# Use -q for a quieter installation
%pip install -q openmm
%pip install -q mdtraj

# Import the libraries we will use
import openmm as mm
from openmm import app, unit
import mdtraj as md
import matplotlib.pyplot as plt
import numpy as np
from sys import stdout
from pdbfixer import PDBFixer # Import PDBFixer directly

print("--> Libraries installed successfully.")


# ===================================================================
# === Step 2: Prepare the Molecular System
# ===================================================================
print("\n--> Step 2: Preparing the molecular system...")

# Download a sample RNA hairpin structure (PDB ID: 2KOC) as a stand-in
!wget -q -O rna_hairpin.pdb https://files.rcsb.org/download/2KOC.pdb

# Use PDBFixer to prepare the molecule (e.g., add missing hydrogen atoms)
fixer = PDBFixer(filename='rna_hairpin.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True) # Remove non-RNA atoms
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0) # Assume a pH of 7.0

# Define the force field (physics rules) for RNA and water
forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

# Create the system by adding a solvent (water) box around the RNA
modeller = app.Modeller(fixer.topology, fixer.positions)
modeller.addSolvent(forcefield, model='tip3pfb', padding=1.0*unit.nanometer)

# Create the final system for OpenMM to simulate
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME,
                                 nonbondedCutoff=1.0*unit.nanometer, constraints=app.HBonds)

print("--> RNA structure has been prepared and solvated in a water box.")


# ===================================================================
# === Step 3: Configure and Run the MD Simulation
# ===================================================================
print("\n--> Step 3: Configuring and running the simulation...")

# Configure the simulation settings (integrator, platform)
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
platform = mm.Platform.getPlatformByName('CUDA') # Use the GPU
properties = {'CudaPrecision': 'mixed'} # Use mixed precision for performance
simulation = app.Simulation(modeller.topology, system, integrator, platform, properties)
simulation.context.setPositions(modeller.positions)

# --- 1. Minimize Energy ---
print('Minimizing energy of the system...')
simulation.minimizeEnergy()

# --- 2. Equilibrate the System ---
print('Equilibrating the system at 300K...')
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
simulation.step(5000) # Run equilibration for 10 ps

# --- 3. Run Production Simulation ---
print('Running Production MD...')
# Set up reporters to save the trajectory (the "movie") and log data
simulation.reporters.append(app.DCDReporter('trajectory_undamaged.dcd', 5000))
simulation.reporters.append(app.StateDataReporter(stdout, 5000, step=True,
        potentialEnergy=True, temperature=True, progress=True, remainingTime=True, speed=True, totalSteps=100000))

# Run the production simulation for 200 picoseconds
simulation.step(100000)

print('\n--> Simulation complete!')


# ===================================================================
# === Step 4: Analyze the Results
# ===================================================================
print("\n--> Step 4: Analyzing the simulation trajectory...")

# Load the trajectory file we just created
traj_undamaged = md.load('trajectory_undamaged.dcd', top=app.PDBFile(fixer.topology, fixer.positions).topology)

# Calculate the Root Mean Square Deviation (RMSD) from the starting frame
rmsd_undamaged = md.rmsd(traj_undamaged, traj_undamaged, frame=0)

# Plot the RMSD over time
plt.figure(figsize=(12, 7))
plt.plot(traj_undamaged.time, rmsd_undamaged, label='Undamaged RNA (Cautious Strategy)')
plt.xlabel('Time (ps)', fontsize=12)
plt.ylabel('RMSD (nm)', fontsize=12)
plt.title('Structural Stability of the RNA Molecule', fontsize=14)
plt.legend()
plt.grid(True)
plt.show()

print("\n--> Analysis complete. A stable RMSD indicates the structure is not unfolding.")

--> Step 1: Installing necessary libraries...


ModuleNotFoundError: No module named 'pdbfixer'

In [ ]:
!conda install -c conda-forge pdbfixer -y

/bin/bash: line 1: conda: command not found
